# Import Libraries

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mping
from skimage import exposure
from skimage.util import random_noise
from skimage import transform
import glob
import random
from skimage.transform import resize

# Build Zoom function

build from https://stackoverflow.com/questions/37119071/scipy-rotate-and-zoom-an-image-without-changing-its-dimensions/37121993#37121993

In [2]:
import numpy as np
from scipy.ndimage import zoom


def clipped_zoom(img, zoom_factor, **kwargs):

    h, w = img.shape[:2]

    # For multichannel images we don't want to apply the zoom factor to the RGB
    # dimension, so instead we create a tuple of zoom factors, one per array
    # dimension, with 1's for any trailing dimensions after the width and height.
    zoom_tuple = (zoom_factor,) * 2 + (1,) * (img.ndim - 2)

    # Zooming out
    if zoom_factor < 1:

        # Bounding box of the zoomed-out image within the output array
        zh = int(np.round(h * zoom_factor))
        zw = int(np.round(w * zoom_factor))
        top = (h - zh) // 2
        left = (w - zw) // 2

        # Zero-padding
        out = np.zeros_like(img)
        out[top:top+zh, left:left+zw] = zoom(img, zoom_tuple, **kwargs)

    # Zooming in
    elif zoom_factor > 1:

        # Bounding box of the zoomed-in region within the input array
        zh = int(np.round(h / zoom_factor))
        zw = int(np.round(w / zoom_factor))
        top = (h - zh) // 2
        left = (w - zw) // 2

        out = zoom(img[top:top+zh, left:left+zw], zoom_tuple, **kwargs)

        # `out` might still be slightly larger than `img` due to rounding, so
        # trim off any extra pixels at the edges
        trim_top = ((out.shape[0] - h) // 2)
        trim_left = ((out.shape[1] - w) // 2)
        out = out[trim_top:trim_top+h, trim_left:trim_left+w]

    # If zoom_factor == 1, just return the input array
    else:
        out = img
    return out

# Augmentation

In [3]:
def skin_aug(src, output):
    for i, img in enumerate(glob.glob((src))):
        """
        Operation
        """
        #original
        image = mping.imread(img)
        #image = resize(image, (224, 224))
        
        #flip
        horizontal = np.fliplr(image)
        vertical = np.flipud(image)
        allflip = np.flipud(horizontal)
        
        #zoom
        zoom70 = clipped_zoom(image, 0.7)
        zoom80 = clipped_zoom(image, 0.8)
        zoom90 = clipped_zoom(image, 0.9)
        zoom110 = clipped_zoom(image, 1.1)
        zoom120 = clipped_zoom(image, 1.2)
        zoom130 = clipped_zoom(image, 1.3)
        
        #rotate
        rotate90 = transform.rotate(image, 90)
        rotate270 = transform.rotate(image, 270)
        rd_rotate = transform.rotate(image, random.uniform(0, 360))
        
        #light
        bright = exposure.adjust_gamma(image, gamma=0.5, gain=1)
        dark = exposure.adjust_gamma(image, gamma=1.5, gain=1)
        z80_bright = exposure.adjust_gamma(zoom80, gamma=0.5, gain=1)
        z120_bright = exposure.adjust_gamma(zoom120, gamma=0.5, gain=1)
        z80_dark = exposure.adjust_gamma(zoom80, gamma=1.5, gain=1)
        z120_dark = exposure.adjust_gamma(zoom120, gamma=1.5, gain=1)
        
        #noise
        rd_noise = random_noise(image) 
        noise_bright = random_noise(bright) 
        noise_dark = random_noise(dark)
        noise_allflip = random_noise(allflip)
        noise_rd_rotate = random_noise(rd_rotate)
        noise_z120 = random_noise(zoom120)
        
        """
        Save Images
        """
        
        #original
        plt.imsave(output.format('o', i), image)
        
        #flip
        plt.imsave(output.format('h', i), horizontal)
        plt.imsave(output.format('v', i), vertical)
        plt.imsave(output.format('af', i), allflip)
        
        #zoom
        plt.imsave(output.format('z70', i), zoom70)
        plt.imsave(output.format('z80', i), zoom80)
        plt.imsave(output.format('z90', i), zoom90)
        plt.imsave(output.format('z110', i), zoom110)
        plt.imsave(output.format('z120', i), zoom120)
        plt.imsave(output.format('z130', i), zoom130)
        
        #rotate
        plt.imsave(output.format('rt90', i), rotate90)
        plt.imsave(output.format('rt270', i), rotate270)
        plt.imsave(output.format('rdrt', i), rd_rotate)
        
        #light
        plt.imsave(output.format('b', i), bright)
        plt.imsave(output.format('d', i), dark)
        plt.imsave(output.format('z80b', i), z80_bright)
        plt.imsave(output.format('z120b', i), z120_bright)
        plt.imsave(output.format('z80d', i), z80_dark)
        plt.imsave(output.format('z120d', i), z120_dark)
        
        #noise
        plt.imsave(output.format('rdn', i), rd_noise)
        plt.imsave(output.format('nb', i), noise_bright)  
        plt.imsave(output.format('nd', i), noise_dark)
        plt.imsave(output.format('naf', i), noise_allflip)
        plt.imsave(output.format('nrt', i), noise_rd_rotate)
        plt.imsave(output.format('nz120', i), noise_z120)

# Atopic Dermatitis

In [22]:
ad_src_path = '/Users/waranthornchansawang/Desktop/RASH_INSPECTOR/Dataset/ORGAN/step3_resized_data/leg/AD/*.jpg'
ad_out_path = '/Users/waranthornchansawang/Desktop/RASH_INSPECTOR/Dataset/ORGAN/step4_augmented_data/leg/AD/ad_leg_{}_{}.jpg'

In [23]:
skin_aug(ad_src_path, ad_out_path)

# Contact Dermatitis

In [24]:
other_src_path = '/Users/waranthornchansawang/Desktop/RASH_INSPECTOR/Dataset/ORGAN/step3_resized_data/leg/Other/*.jpg'
other_out_path = '/Users/waranthornchansawang/Desktop/RASH_INSPECTOR/Dataset/ORGAN/step4_augmented_data/leg/Other/other_leg_{}_{}.jpg'

In [25]:
skin_aug(other_src_path, other_out_path)